<a href="https://colab.research.google.com/github/DCA-28/DCA-28/blob/dev_branch/Trabalho_1_filas_e_ru%C3%ADnas_do_apostador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho 1
##Filas e Ruína do Apostador
##Modelagem e Avaliação de Desempenho

## Entrega Preliminar

### Alunos:
Daniel Corcino de Albuquerque - DRE: 118188457

Letícia Freire Carvalho de Sousa - DRE: 118025324

Lucas Favilla Ferreira Alves da Silva - DRE: 119156518

Roberto Leonie Ferreira Moreira - DRE: 116062192



## Simulação

Fizemos uma simulação em que passamos como parâmetros:

$\lambda$ - taxa da variável exponencial que modela o tempo entre as chegadas dos clientes  

$\mu$ - taxa da variável exponencial que modela a vida residual dos clientes

`LIM_TIME` - Tempo limite que passamos aceitando novos clientes na fila de espera

Após passarmos do tempo limite terminamos a simulação com os clientes que já estavam na fila de espera.


Estamos computando os ciclos de vida do servidor (do nascimento a morte) bem como seus ciclos ociosos.

### Métricas

Para entrega preliminar computamos:

  $E(N)$ = valor esperado do número de clientes no sistema, a cada momento

  $E(Nq)$ = número de clientes na fila de espera, a cada chegada de novo cliente

  $E(X)$ = valor esperado do tempo de serviço prestado a cada cliente

  $E(W)$ = valor esperado do tempo na fila de espera de cada cliente

  $E(T)$ = valor esperado do tempo de cada cliente no sistema


In [ ]:
import numpy as np
import random
import copy
import time

# A simulacao em si
def simulation(LAMBDA, MU, LIM_TIME, WAITING_MAX_SIZE = -1, VERBOSE = 0):

  # Medidas interessantes a serem analisadas
  class Measures:
    def __init__(self):
      #variaveis auxiliares para as medidas
      self.TotalClients = 0 # número total de clientes que passaram pelo sistema
      self.sum_Nq = 0 #  somatorio do número de clientes na fila de espera a cada vez que um cliente chega
      self.N = 0 # total medio de clientes no sistema a cada momento
      self.T = 0 # somatorio do tempo total gasto no sistema por cada cliente
      self.W = 0 # somatorio do tempo total gasto na fila de espera por cada cliente
      self.X = 0 # somatorio do tempo total gasto em serviço por cada cliente

      self.count_idle_cycles = 0 # numero de ciclos ociosos
      self.count_birth_and_death_processes = 0 # numero de ciclos de vida e morte
      self.sum_idle_cycles_time = 0 # somatorio da duração de cada ciclo ocioso
      self.sum_birth_and_death_processes_time = 0 # somatorio da duração de cada ciclo de vida e morte
      self.sum_time = 0 # tempo total de simulação

    # valor esperado do número de clientes no sistema, a cada momento
    def Expected_Value_N(self):
      return self.N / (self.sum_time)

    # valor esperado do número de clientes na fila de espera, a cada chegada de novo cliente
    def Expected_Value_Nq(self):
      return self.sum_Nq / self.TotalClients

    # valor esperado da porcentagem do tempo em que o sistema está ativo
    def Expected_Value_Rho(self):
      return self.sum_birth_and_death_processes_time / (self.sum_time)

    # valor esperado do tempo de serviço prestado a cada cliente
    def Expected_Value_X(self):
      return self.X / self.TotalClients

    # valor esperado do tempo na fila de espera de cada cliente
    def Expected_Value_W(self):
      return self.W / self.TotalClients

    # valor esperado do tempo de cada cliente no sistema
    def Expected_Value_T(self):
      return self.T / self.TotalClients

  # classe que modela os eventos de chegada e saída de cada cliente no sistema
  # id -- id do cliente -- incremental a partir de 1
  # event_type -- tipo do evento ("A" - chegada, "D" - saída)
  # arrival_time -- momento da chegada no sistema
  # departure_time -- momento de saída do sistema
  # service_start -- momento em que começa a ser serviço
  # time -- momento atual, pode ser igual a arrival_time ou departure_time, a depender
  #         de qual evento está sendo modelado

  class Event:
    def __init__(self, event_type, time, id):
      self.event_type = event_type
      self.time = time
      self.service_start = -1
      if(event_type == 'A'):
        self.arrival_time = self.time
        self.departure_time = -1
      else:
        self.departure_time = self.time
      self.id = id

    # para imprimir de forma expressiva os dados do cliente
    def __str__(self):
      if(self.event_type == "A"):
        return str(self.id) + "\t|\t"+str(f"{self.arrival_time:.3f}").format()+ "\t|"
      return str(self.id) + "\t|\t" + str(f"{self.arrival_time:.3f}") + "\t|\t"
      + str(f"{self.service_start:.3f}") + "\t|\t" + str(f"{self.departure_time:.3f}") + "\t|"

    # comparador, utilizado para sabermos que evento vem primeiro
    # em caso de empate damos prioridade para a saída do sistema
    def __lt__(self, other):
      if(self.time<other.time):
          return True
      if(self.time>other.time):
          return False
      else:
          return (self.event_type == 'D')

  # o tempo de vida residual é modelado por uma variavel aleatória exponencial com taxa MU
  def residual_life():
    return random.expovariate(MU)

  # o momento da proxima chegada é modelado pela soma de variaveis aleatórias exponenciais iid com taxa LAMBDA
  ARRIVAL_TIME = 0
  def next_arrival():
    return ARRIVAL_TIME + random.expovariate(LAMBDA)

  # classe para modelar o sistema
  # waiting_queue -- lista que representa a fila de espera (células são do tipo Event)
  # server -- variável do tipo Event que representa o cliente que está sendo servido
  class System:
    def __init__(self, waiting_max_size = -1):
      self.waiting_queue = []
      self.server = None
      self.waiting_max_size = waiting_max_size

    # cliente e entra na lista de espera
    def arrival(self, e):
      if(self.waiting_max_size == -1 or self.waiting_queue_size() < self.waiting_max_size):
        self.waiting_queue.append(e)

    # cliente que está sendo servido sai do sistema
    # garantidamente só é chamado se o sistema estava ocupado
    def departure(self):
      server = copy.deepcopy(self.server)
      self.server = None
      return server

    # o próximo cliente sai da fila de espera e é servido
    def next(self, T):
      if(self.busy() or not(self.waiting_queue)):
        return None
      self.server = self.waiting_queue.pop(0)
      x = residual_life()
      measures.X += x
      self.server.departure_time = T + x
      self.server.time = self.server.departure_time
      return self.server

    # verifica se o sistema está ocupado
    def busy(self):
      return not(self.server == None)

    # tamanho da fila de espera
    def waiting_queue_size(self):
      return len(self.waiting_queue)

    # tamanho da fila (fila de espera + servidor)
    def full_queue_size(self):
      s = len(self.waiting_queue)
      if(self.busy()):
        s += 1
      return s

    # para imprimir de forma expressiva os dados dos clientes na fila
    # imprimimos os dados de no máximo 5 clientes para não poluir a simulação
    def __str__(self):
      ans = "\tServidor: \n"
      ans += "\033[1m"+"\tid\t|\tchegada\t|\tservidor|\tsaída\t|\n"+"\033[0m"
      ans += "\t" + str(self.server) + "\n"


      ans += "\n \tFila de Espera: ("+ str(self.waiting_queue_size())+ ") \n"

      ans += "\033[1m"+"\tid\t|\tchegada\t|\n"+"\033[0m"

      lim = 5
      for w in self.waiting_queue:
        ans += "\t" + str(w) + " \n"
        lim -= 1
        if(lim == 0):
          ans += "\t...\n"
          break
      ans += "---------------------------------------\n"
      return ans


  # variáveis principais
  system = System(WAITING_MAX_SIZE)
  measures = Measures()

  id = 0 # id do próximo cliente que chegará

  ARRIVAL_TIME = 0
  # evento de chegada do próximo cliente
  # é inicializado fora do laço principal para que a simulção comece com um cliente no sistema
  next_arrival_event = Event('A', ARRIVAL_TIME, id)

  # evento de saída do cliente que está no servidor
  # None se o sistema está ocioso, ou antes da primeira iteração do laço principal
  next_departure_event = None

  T = -1 # tempo atual
  birth = -1 # tempo do último nascimento, -1 se ocioso
  death = -1 # tempo da última morte, -1 se é o primeiro ciclo de vida e morte
  count_clients = 0 # número total de clientes que passaram pelo sistema no ciclo de vida e morte atual

  # laço principal da simulação
  # Quando a simulação chega ao tempo limite paramos de aceitar a chegada de novos clientes
  # e, após, a simulação para depois de atender todos os clientes que já estavam na fila
  while(next_departure_event or next_arrival_event):
    # evento atual, pode ser de chegada ou de saída
    current_event = []

    # se o evento atual é de chegada
    if(not(next_departure_event) or (not(next_arrival_event == None) and next_arrival_event.time < next_departure_event.time) ):
      if(VERBOSE > 1):
        print("Arrival event: ")

      current_event = next_arrival_event
      id += 1
      next_arrival_event = None
      # aceitamos nova chegada na fila de espera se não terminou o tempo da simulação
      if(T < LIM_TIME):
        ARRIVAL_TIME = next_arrival()
        next_arrival_event = Event('A', ARRIVAL_TIME, id)

    # se o evento atual é de partida
    else:
      if(VERBOSE > 1):
        print("Departure event: ")
      current_event = next_departure_event

      # atualizamos a métrica do tempo total do cliente no sistema
      measures.T += next_departure_event.departure_time - next_departure_event.arrival_time

      next_departure_event = None

    if(VERBOSE > 1):
      print("\033[1m"+"id\t|\tchegada\t|\tservidor|\tsaída\t|\n"+"\033[0m")
      print(current_event)

    # atualizamos tempo atual da simulação
    last_T = T
    T = current_event.time

    if(VERBOSE > 1):
      print(f"\nCurrent event time: {T:.3f}\n")

    # se o evento atual é de chegada e vai ser aceito
    if(current_event.event_type == 'A' and not(system.waiting_queue_size() == system.waiting_max_size)):
      # atualizamos a métrica do número de clientes na fila de espera no momento de chegada do cliente
      measures.sum_Nq += system.waiting_queue_size()

      count_clients += 1
      if(birth == -1):
        birth = T

        # se estamos começando novo ciclo de vida e morte
        if(not(death == -1)):
          # atualizamos várias métricas:

          # somatório do número de clientes no sistema a cada momento
          measures.N += system.full_queue_size() * (T - last_T)

          # número de ciclos ociosos
          measures.count_idle_cycles += 1

          # somatorio da duração de cada ciclo ocioso
          measures.sum_idle_cycles_time += birth - death

          # tempo total de simulação
          measures.sum_time += birth - death

          if(VERBOSE > 1):
            print(f"servidor ocioso de {death:.3f} ate {birth:.3f}")
      system.arrival(current_event)

    # se o evento atual é de saída
    else:
      # somatório do número de clientes no sistema a cada momento
      measures.N += system.full_queue_size() * (T - last_T)

      # cliente termina de ser servido
      system.departure()

    serving = system.next(T)
    # se um cliente começou a ser servido neste momento
    if(serving):
      # somatório do tempo que cada cliente passa na fila de espera
      measures.W += T - serving.arrival_time

      # gerando o evento de saída do cliente do sistema
      serving.service_start = T
      serving.event_type = 'D'
      next_departure_event = serving

    # se o sistema está ocioso
    if(not(system.busy())):
      death = T
      # atualizamos várias métricas:

      # número de ciclos de vida e morte
      measures.count_birth_and_death_processes += 1

      # somatorio da duração de cada ciclo de vida e morte
      measures.sum_birth_and_death_processes_time += death - birth

      # tempo total de simulação
      measures.sum_time += death - birth

      # numero total de clientes que passaram pelo sistema
      measures.TotalClients += count_clients

      if(VERBOSE > 1):
        print(f"servidor ficou ativo de {birth:.3f} ate {death:.3f} e atendeu {count_clients} clientes")
      count_clients = 0
      birth = -1

      if(VERBOSE > 1):
        print("")
    if(VERBOSE > 1):
      print(system)

  if(VERBOSE > 0):
    print("Métricas obtidas:")
    print(f"E[T] =  {measures.Expected_Value_T():.3f}")
    print(f"E[W] =  {measures.Expected_Value_W():.3f}")
    print(f"E[X] =  {measures.Expected_Value_X():.3f}")
    print("")
    print(f"E[N] =  {measures.Expected_Value_N():.3f}")
    print(f"E[Nq] =  {measures.Expected_Value_Nq():.3f}")
    print(f"E[Rho] =  {measures.Expected_Value_Rho():.3f}")

  return measures


## Resultado das simulações

In [ ]:
def experiment(LAMBDA, MU, WAITING_MAX_SIZE = -1, MAX_ITERATIONS = 5):
  LIM_TIME = 100000
  T = 1.96

  averages = {"T": 0.0, "W": 0.0, "X":0.0, "N":0.0, "Nq": 0.0, "Rho": 0.0}
  variances = {"T": 0.0, "W": 0.0, "X":0.0, "N":0.0, "Nq": 0.0, "Rho": 0.0}
  std_dev = {"T": 0.0, "W": 0.0, "X":0.0, "N":0.0, "Nq": 0.0, "Rho": 0.0}
  results = {"T": [], "W": [], "X":[], "N":[], "Nq":[], "Rho": []}
  confidence_interval = {"T": [], "W": [], "X":[], "N":[], "Nq":[], "Rho": []}


  def calc_average(s):
    n = len(s)
    total = 0
    for x in s:
      total += x
    return total / n

  def calc_variance(s, average):
    n = len(s)
    if(n == 1):
      return 0.0
    total = 0
    for x in s:
      total += (x - average) ** 2
    return total / (n - 1)


  def calc_confidence_interval(t):
    return [averages[key] - t * std_dev[key] / ((it+1)**0.5), averages[key] + t * std_dev[key] / ((it+1)**0.5)]

  for it in range(MAX_ITERATIONS):
    current = simulation(LAMBDA, MU, WAITING_MAX_SIZE, LIM_TIME)
    results["T"].append(current.Expected_Value_T())
    results["W"].append(current.Expected_Value_W())
    results["X"].append(current.Expected_Value_X())
    results["N"].append(current.Expected_Value_N())
    results["Nq"].append(current.Expected_Value_Nq())
    results["Rho"].append(current.Expected_Value_Rho())

    print(f"Simulacao {it+1}")
    for key, value in results.items():
      averages[key] = calc_average(value)
      variances[key] = calc_variance(value, averages[key])
      std_dev[key] = variances[key] ** (1/2)
      confidence_interval[key] = calc_confidence_interval(T)
      print(f" {key} média {averages[key]:.3f} variancia {variances[key]} desvio padrão {std_dev[key]} intervalo de confiança: ({confidence_interval[key][0]:.3f}, {confidence_interval[key][1]:.3f})")

    print()




#Resultado dos experimentos

## Filas infinitas

Abaixo apresentamos o resultado dos experimentos, cada um com 5 simulações com os valores sugeridos no enunciado e LIM_TIME = 100000.

In [ ]:
experiment(1, 2)

Simulacao 1
 T média 1.235 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.235, 1.235)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 1.235 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.235, 1.235)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 Rho média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)

Simulacao 2
 T média 0.706 variancia 0.5593864799164623 desvio padrão 0.7479214396689416 intervalo de confiança: (-0.330, 1.743)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.706 variancia 0.5593864799164623 desvio padrão 0.7479214396689416 intervalo de confiança: (-0.330, 1.743)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 in

In [ ]:
experiment(2, 4)

Simulacao 1
 T média 0.125 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.125, 0.125)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.125 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.125, 0.125)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 Rho média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)

Simulacao 2
 T média 0.108 variancia 0.0005963291486381708 desvio padrão 0.024419851527766726 intervalo de confiança: (0.074, 0.142)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.108 variancia 0.0005963291486381708 desvio padrão 0.024419851527766726 intervalo de confiança: (0.074, 0.142)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrã

In [ ]:
experiment(1.05, 1)

Simulacao 1
 T média 0.672 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.672, 0.672)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.672 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.672, 0.672)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 Rho média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)

Simulacao 2
 T média 0.363 variancia 0.19095302973379752 desvio padrão 0.43698172700216825 intervalo de confiança: (-0.242, 0.969)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.363 variancia 0.19095302973379752 desvio padrão 0.43698172700216825 intervalo de confiança: (-0.242, 0.969)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.

In [ ]:
experiment(1.10, 1)

Simulacao 1
 T média 0.853 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.853, 0.853)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.853 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.853, 0.853)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 Rho média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)

Simulacao 2
 T média 0.705 variancia 0.04395597111751695 desvio padrão 0.209656793635496 intervalo de confiança: (0.414, 0.995)
 W média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 X média 0.705 variancia 0.04395597111751695 desvio padrão 0.209656793635496 intervalo de confiança: (0.414, 0.995)
 N média 1.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.000, 1.000)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 inte

## Filas Finitas

Abaixo apresentamos o resultado dos experimentos, cada um com 5 simulações com os valores sugeridos no enunciado e LIM_TIME = 100000.

In [ ]:
experiment(1.05, 1, 5)

Simulacao 1
 T média 1.531 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.531, 1.531)
 W média 0.573 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.573, 0.573)
 X média 0.958 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.958, 0.958)
 N média 1.203 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.203, 1.203)
 Nq média 0.222 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.222, 0.222)
 Rho média 0.864 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.864, 0.864)

Simulacao 2
 T média 2.359 variancia 1.369729970095523 desvio padrão 1.1703546343290665 intervalo de confiança: (0.737, 3.981)
 W média 1.367 variancia 1.258381959458829 desvio padrão 1.1217762519588428 intervalo de confiança: (-0.188, 2.921)
 X média 0.992 variancia 0.002359859233707634 desvio padrão 0.048578382370223425 intervalo de confiança: (0.925, 1.060)
 N média 1.564 variancia 0.2608000531535664 desvio padrão 0.5106858654335035 intervalo de confiança: (0.8

In [ ]:
experiment(1.10, 1, 5)

Simulacao 1
 T média 1.377 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (1.377, 1.377)
 W média 0.420 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.420, 0.420)
 X média 0.956 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.956, 0.956)
 N média 0.867 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.867, 0.867)
 Nq média 0.000 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.000, 0.000)
 Rho média 0.800 variancia 0.0 desvio padrão 0.0 intervalo de confiança: (0.800, 0.800)

Simulacao 2
 T média 1.328 variancia 0.004706740028990375 desvio padrão 0.06860568510692372 intervalo de confiança: (1.233, 1.423)
 W média 0.381 variancia 0.0031043132237277143 desvio padrão 0.055716364056960094 intervalo de confiança: (0.304, 0.458)
 X média 0.947 variancia 0.00016613459712902742 desvio padrão 0.012889321049963315 intervalo de confiança: (0.929, 0.965)
 N média 0.966 variancia 0.019696335410947136 desvio padrão 0.1403436333110524 intervalo de co

# Espaço de Teste

Abaixo temos uma célula em que podemos testar diferentes valores de $\mu$, $\lambda$ e LIM_TIME.

Além disso, há um parâmetro extra booleano VERBOSE.

Quando passamos VERBOSE = 2 nós mostramos cada evento de chegada e saída de clientes do sistema, além dos resultados das métricas calculadas.

Quando passamos VERBOSE = 1 nós mostramos somente resultados das métricas calculadas.


Quando passamos VERBOSE = 0 não mostramos nada.


In [ ]:
print("   SIMULAÇÃO   ")
LAMBDA = float(input("Taxa lambda (momentos de chegada) : "))
MU = float(input("Taxa mu (vida residual) : "))
LIM_TIME = float(input("Tempo máximo aceitando novos clientes (s): "))
WAITING_MAX_SIZE = int(input("Tamanho máximo da fila de espera (-1, se ilimitada)"))
VERBOSE = int(input("Verbose? (0 - Não imprime nada; 1 - Imprime métricas obtidas; 2 - Imprime a simulação dos eventos e métricas obtidas)"))
simulation(LAMBDA, MU, LIM_TIME, WAITING_MAX_SIZE, VERBOSE)

   SIMULAÇÃO   
Taxa lambda (momentos de chegada) : 1.05
Taxa mu (vida residual) : 1
Tempo máximo aceitando novos clientes (s): 15
Tamanho máximo da fila de espera (-1, se ilimitada)2
Verbose? (0 - Não imprime nada; 1 - Imprime métricas obtidas; 2 - Imprime a simulação dos eventos e métricas obtidas)1
Métricas obtidas:
E[T] =  1.227
E[W] =  0.448
E[X] =  0.779

E[N] =  0.795
E[Nq] =  0.071
E[Rho] =  0.622


<__main__.simulation.<locals>.Measures at 0x7fe0846da9b0>